# Perceptrón aplicado a iris

**Lectura del corpus:** $\;$ comprobamos también que las matrices de datos y etiquetas tienen las filas y columnas que toca

In [ ]:
# Importar la biblioteca numpy y asignarle el alias 'np' para facilitar su uso posterior.
import numpy as np


# Importar la función 'load_iris' del módulo 'datasets' de la biblioteca 'sklearn'.
from sklearn.datasets import load_iris

# Cargar el conjunto de datos Iris y almacenarlo en la variable 'iris'.
iris = load_iris()

# Acceder a los datos de características (features) del conjunto de datos Iris, y convertirlos a 'float16'
X = iris.data.astype(np.float16)

# Acceder a las etiquetas (objetivos), convertirlas al tipo de dato 'uint' y remodelarlas para que sean una matriz de una columna
y = iris.target.astype(np.uint).reshape(-1, 1)

# Imprimir la forma (dimensiones) de las matrices 'X' y 'y', seguido de las primeras 5 filas
print(X.shape, y.shape, "\n", np.hstack([X, y])[:5, :])

**Partición del corpus:** $\;$ Creamos un split de iris con un $20\%$ de datos para test y el resto para entrenamiento (training), barajando previamente los datos de acuerdo con una semilla dada para la generación de números aleatorios. Aquí, como en todo código que incluya aleatoriedad (que requiera generar números aleatorios), conviene fijar dicha semilla para poder reproducir experimentos con exactitud.

In [ ]:
# Importar la función 'train_test_split' del módulo 'model_selection' de la biblioteca 'sklearn'.
from sklearn.model_selection import train_test_split

# Dividir los conjuntos de datos de características (X) y etiquetas (y) en subconjuntos de entrenamiento y prueba.
# El parámetro 'test_size=0.2' indica que el 20% de los datos se utilizará como conjunto de prueba, el 80% restante será el conjunto de entrenamiento.
# El parámetro 'shuffle=True' asegura que los datos se barajen antes de dividirlos, lo cual es importante para evitar sesgos.
# El parámetro 'random_state=23' se utiliza para garantizar la reproducibilidad del proceso de división.
# Los conjuntos de entrenamiento y prueba resultantes se almacenan en 'X_train', 'X_test', 'y_train' y 'y_test', respectivamente.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)

# Imprimir las dimensiones de los conjuntos de entrenamiento y prueba de las características (X), mostrando información sobre cuántas muestras hay en cada conjunto.
print(X_train.shape, X_test.shape)

**Implementación de Perceptrón:** $\;$ devuelve pesos en notación homogénea, $\mathbf{W}\in\mathbb{R}^{(1+D)\times C};\;$ también el número de errores e iteraciones ejecutadas

In [ ]:
def perceptron(X, y, b=0.1, a=1.0, K=200):
    # X: Matriz de características de entrada.
    # y: Vector de etiquetas.
    # b: Margen de clasificación.
    # a: Tasa de aprendizaje.
    # K: Número máximo de iteraciones.

    N, D = X.shape  # N: Número de muestras, D: Número de características.
    Y = np.unique(y)  # Y: Etiquetas únicas en el conjunto de datos.
    C = Y.size  # C: Número de clases únicas.
    W = np.zeros((1+D, C))  # W: Matriz de pesos inicializada a ceros.

    for k in range(1, K+1):  # Bucle sobre las iteraciones.
        E = 0  # E: Contador de errores en la clasificación.

        for n in range(N):  # Bucle sobre todas las muestras.
            xn = np.array([1, *X[n, :]])  # xn: Vector de características con un término de sesgo añadido.
            cn = np.squeeze(np.where(Y==y[n]))  # cn: Índice de la clase actual de la muestra.
            gn = W[:,cn].T @ xn  # gn: Producto punto de la muestra con los pesos de su clase.
            err = False  # err: Indicador de error en la clasificación.

            # Bucle para actualizar los pesos si se encuentra un error.
            for c in np.arange(C):
                if c != cn and W[:,c].T @ xn + b >= gn:
                    W[:, c] = W[:, c] - a*xn  # Actualizar pesos para clases incorrectas.
                    err = True

            # Actualizar pesos para la clase correcta si se encontró un error.
            if err:
                W[:, cn] = W[:, cn] + a*xn
                E = E + 1  # Incrementar el contador de errores.

        # Si no hay errores, detener el entrenamiento.
        if E == 0:
            break;

    return W, E, k  # Devolver la matriz de pesos, el número de errores y el número de iteraciones realizadas.

**Aprendizaje de un clasificador (lineal) con Perceptrón:** $\;$ Perceptrón minimiza el número de errores de entrenamiento (con margen)
$$\mathbf{W}^*=\operatorname*{argmin}_{\mathbf{W}=(\boldsymbol{w}_1,\dotsc,\boldsymbol{w}_C)}\sum_n\;\mathbb{Y}\biggl(\max_{c\neq y_n}\;\boldsymbol{w}_c^t\boldsymbol{x}_n+b \;>\; \boldsymbol{w}_{y_n}^t\boldsymbol{x}_n\biggr)$$

In [ ]:
W, E, k = perceptron(X_train, y_train)
print("Número de iteraciones ejecutadas: ", k)
print("Número de errores de entrenamiento: ", E)
print("Vectores de pesos de las clases (en columnas y en notación homogénea):\n", W);

**Cálculo de la tasa de error en test:**

In [ ]:

# Preparar el conjunto de datos de prueba (X_test) para la clasificación.
# Esto se hace añadiendo una columna de unos al inicio de X_test.
# La columna de unos actúa como un término de sesgo en el modelo de perceptrón.
X_testh = np.hstack([np.ones((len(X_test), 1)), X_test])

# Realizar la predicción en el conjunto de prueba.
# Esto se hace calculando el producto punto (usando el operador '@') entre X_testh y la matriz de pesos W.
# La función 'np.argmax' se usa para seleccionar el índice de la clase con el valor más alto en cada fila,
# lo que corresponde a la clase predicha por el modelo.
# Los resultados se reforman en una matriz de una columna.
y_test_pred  = np.argmax(X_testh @ W, axis=1).reshape(-1, 1)

# Calcular la tasa de error de las predicciones.
# Esto se hace contando el número de veces que las predicciones (y_test_pred) difieren de las etiquetas verdaderas (y_test),
# y dividiendo este número por la longitud del conjunto de prueba (X_test).
# El resultado es la proporción de predicciones incorrectas.
err_test = np.count_nonzero(y_test_pred != y_test) / len(X_test)

# Imprimir la tasa de error en el conjunto de prueba.
# El resultado se muestra como un porcentaje con un decimal.
print(f"Tasa de error en test: {err_test:.1%}")


**Ajuste del margen:** $\;$ experimento para aprender un valor de $b$

In [ ]:
# Bucle sobre diferentes valores del parámetro de margen 'b'.
for b in (.0, .01, .1, 10, 100):
    # Entrenar el perceptrón con el conjunto de entrenamiento (X_train, y_train), utilizando el valor actual de 'b' y un límite máximo de 1000 iteraciones.
    # La función 'perceptron' devuelve la matriz de pesos 'W', el número de errores 'E' y el número de iteraciones 'k'.
    W, E, k = perceptron(X_train, y_train, b=b, K=1000)

    # Imprimir el valor actual de 'b', el número de errores 'E', y el número de iteraciones 'k' para cada entrenamiento.
    print(b, E, k)

**Interpretación de resultados:** $\;$ los datos de entrenamiento no parecen linealmente separables; no está claro que un margen mayor que cero pueda mejorar resultados, sobre todo porque solo tenemos $30$ muestras de test; con margen .1 ya hemos visto que se obtiene un error (en test) del $16.7\%$

Iteramos con b y calculamos error

In [ ]:
for b in (.0, .01, .1, 10, 50):

    W, E, k = perceptron(X_train, y_train, b=b, K=1000)

    X_testh = np.hstack([np.ones((len(X_test), 1)), X_test])

    y_test_pred  = np.argmax(X_testh @ W, axis=1).reshape(-1, 1)

    err_test = np.count_nonzero(y_test_pred != y_test) / len(X_test)

    print(f"Iteracion con b: {b} Tasa de error en test: {err_test:.1%}")

Iteramos con a y calculamos error

In [ ]:
for a in ( .01, .1, 10, 100):

    W, E, k = perceptron(X_train, y_train, b=50, a=a, K=1000)

    X_testh = np.hstack([np.ones((len(X_test), 1)), X_test])

    y_test_pred  = np.argmax(X_testh @ W, axis=1).reshape(-1, 1)

    err_test = np.count_nonzero(y_test_pred != y_test) / len(X_test)

    print(f"Iteracion con a: {a} Tasa de error en test: {err_test:.1%}")

Iteramos con el numero máximo de iteraciones y calculamos error

In [ ]:
for k in ( 200, 500, 1000, 10000):

    W, E, k = perceptron(X_train, y_train, a=0.01, b=50, K=k)

    X_testh = np.hstack([np.ones((len(X_test), 1)), X_test])

    y_test_pred  = np.argmax(X_testh @ W, axis=1).reshape(-1, 1)

    err_test = np.count_nonzero(y_test_pred != y_test) / len(X_test)

    print(f"Iteracion con k: {k} Tasa de error en test: {err_test:.1%}")

¿Qué combinación de valores de a, b y k seleccionarias? Justifica la respuesta